In [3]:
import numpy as np
from scipy.linalg import null_space

def caratheodory(P, w):
    n = P.shape[0]
    d = 1

    if n <= d + 1:
        return P, w 

    unique_points = np.unique(P)
    if len(unique_points) <= d + 1:
        print(f"Not enough unique points: {unique_points}")
        return P, w

    A = []
    for i in range(1, n):
        A.append(P[i] - P[0]) 
    A = np.array(A)

    if A.ndim == 1:
        A = A[np.newaxis, :]

    print("Matrix A:\n", A)
    print("Shape of A:", A.shape)

    v = null_space(A.T)
    print("Null Space:\n", v)

    if v.size == 0:
        raise ValueError("Null space is empty; A may not have sufficient rank.")

    v = v[:, 0]
    v1 = -np.sum(v)
    v = np.append(v1, v)

    alpha = np.inf

    for i in range(n):
        if v[i] > 0:
            alpha = min(alpha, w[i] / v[i])

    S = []
    u = []
    for i in range(n):
        if (w[i] - alpha * v[i]) > 0:
            S.append(P[i])
            u.append(w[i] - alpha * v[i])

    S = np.array(S)
    u = np.array(u)

    if len(P) > d + 1:
        return caratheodory(S, u)
    return S, u

P = np.array([1, 3, 5, 7, 9])
w = np.array([0.25, 0.25, 0.25, 0.15, 0.10])
U = np.sqrt(w)

weightedP = np.multiply(P, U)

x = 3 
v = weightedP * x 

squaredl2Normofv = np.linalg.norm(v)**2
print("Squared L2 Norm of weightedP:", squaredl2Normofv)

try:
    S, u = caratheodory(P, w)
    print("S:", S)
    print("u:", u)
except ValueError as e:
    print(e)


Squared L2 Norm of weightedP: 217.8
Matrix A:
 [[2 4 6 8]]
Shape of A: (1, 4)
Null Space:
 []
Null space is empty; A may not have sufficient rank.


In [4]:
import numpy as np

def Caratheodory(P, u, dtype='float64'):
    while True:
        n = np.count_nonzero(u)
        d = P.shape[1]
        u_non_zero = np.nonzero(u)[0]

        if n <= d + 1:
            return u

        A = P[u_non_zero]
        reduced_vec = np.outer(A[0], np.ones(A.shape[0] - 1, dtype=dtype))
        A = A[1:].T - reduced_vec

        _, _, V = np.linalg.svd(A, full_matrices=True)
        v = V[-1]
        v = np.insert(v, 0, -1 * np.sum(v))

        idx_good_alpha = np.nonzero(v > 0)[0]
        alpha = np.min(u[u_non_zero][idx_good_alpha] / v[idx_good_alpha])

        w = np.zeros_like(u, dtype=dtype)
        tmp_w = u[u_non_zero] - alpha * v
        tmp_w[np.argmin(tmp_w)] = 0.0
        w[u_non_zero] = tmp_w
        u = w

P = np.array([[1, 2], [2, 3], [9, 4], [6, 5], [5, 3], [8, 4]]) 
w = np.array([0.25, 0.25, 0.25, 0.15, 0.05, 0.05]) 
U = np.sqrt(w)

weightedP = np.multiply(P, U[:, None])

x = np.array([3])
v = []

for i in range(len(weightedP)):
    v.append(weightedP[i] * x)

squaredl2Normofv = np.linalg.norm(v) ** 2
print("Squared L2 norm of v:", squaredl2Normofv)

u = Caratheodory(P, w)
print("Reduced weight vector u:", u)

Squared L2 norm of v: 392.40000000000003
Reduced weight vector u: [0.01666667 0.61666667 0.36666667 0.         0.         0.        ]
